In [1]:
from Datos import Datos
from EstrategiaParticionado import *
from Clasificador import *
import matplotlib.pyplot as plt
import numpy as np

<h4>Declaración de funciones generales</h4>

In [2]:
def transformar_datos(dataset):
    
    # Utilizamos el diccionario para adaptar los datos
    for i in range(dataset.datos.shape[0]):
        for j in range(dataset.datos.shape[1]):
            if dataset.nominalAtributos[j]:
                dataset.datos[i, j] = dataset.diccionario[j][dataset.datos[i, j]]
                
    

def get_train_test(dataset, seed=None, p=0.2):
    vs = ValidacionSimple(p, 1)
    particiones = vs.creaParticiones(len(dataset.datos), seed=seed)

    datostrain = dataset.extraeDatos(particiones[0].indicesTrain)
    datostest = dataset.extraeDatos(particiones[0].indicesTest)
    datostrain = datostrain.astype('float32')
    datostest = datostest.astype('float32')
    return datostrain, datostest

<h4>Prueba inicial del funcionamiento del código</h4>

In [3]:
dataset = Datos('titanic.data')

transformar_datos(dataset)
datostrain, datostest = get_train_test(dataset)

ag = AlgoritmoGenetico()
ag.entrenamiento(datostrain, dataset.nominalAtributos, dataset.diccionario, tamanio_poblacion=50, n_epocas=100, reglas_por_ind=5)

Generacion: 0
Fitness medio: 0.49777777777777776
Fitness mejor individuo: 0.604778972520908
###################################################
Generacion: 1
Fitness medio: 0.5055722819593788
Fitness mejor individuo: 0.6033452807646356
###################################################
Generacion: 2
Fitness medio: 0.5022652329749103
Fitness mejor individuo: 0.6062126642771805
###################################################
Generacion: 3
Fitness medio: 0.4991971326164875
Fitness mejor individuo: 0.6078853046594982
###################################################
Generacion: 4
Fitness medio: 0.5154408602150538
Fitness mejor individuo: 0.6088410991636798
###################################################
Generacion: 5
Fitness medio: 0.5410083632019116
Fitness mejor individuo: 0.6088410991636798
###################################################
Generacion: 6
Fitness medio: 0.5530800477897252
Fitness mejor individuo: 0.613142174432497
#############################################

Imprimimos el mejor individuo de la última generación

In [4]:
print(ag.ultimo_individuo)

[[1 1 0 1 0 1 0 0 1 1 1 1 1 1 0 1 1 1 0 0 0 1]
 [0 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 0 0 1 0 1 1]
 [1 1 0 1 0 0 0 0 1 0 1 1 1 1 1 1 1 0 1 1 0 1]
 [1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1]
 [1 1 0 1 0 1 0 0 1 1 1 1 1 1 0 1 1 1 0 0 1 1]]


In [7]:
pred = ag.clasifica(datostest, dataset.nominalAtributos, dataset.diccionario)

Y el porcentaje de error, siendo este menor del 25%.

In [8]:
print(1-ag.error(datostest, pred))

0.7655502392344498


<h1> Apartado 1 </h1>

<h3> a. Generación de la población inicial </h3>

<h3> b. Mecanismo de cruce implementado </h3>

<h3> c. Mecanismo de mutación implementado </h3>

<h3> d. Mecanismo de clasificación implementado </h3>

<h1> Apartado 2 </h1>

<h3> a. Conjunto de datos: tic-tac-toe </h3

In [ ]:
dataset = Datos('tic-tac-toe.data')

transformar_datos(dataset)
datostrain, datostest = get_train_test(dataset)

In [ ]:

ag = AlgoritmoGenetico()
ag.entrenamiento(datostrain, dataset.nominalAtributos, dataset.diccionario, tamanio_poblacion=300, n_epocas=300)

<h3> b. Conjunto de datos: titanic </h3>

<h1> Apartado 3 </h1>

<h3> a. Porcentaje de aciertos frente a número de reglas </h3>

<h3> b. Porcentaje de aciertos frente a número de individuos </h3>

<h3> c. Porcentaje de aciertos frente a número de generaciones </h3>

<h3> d. Porcentaje de aciertos frente a tasa de cruce </h3>

<h3> e. Porcentaje de aciertos frente a tasa de mutación </h3>

<h1> Apartado 4 </h1>

<h3> a. Evolución del fitness del mejor individuo de la población </h3>

<h3> b. Evolución del fitness medio de la población </h3>

<h1> Apartado 5 </h1>